In [36]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as stats

root_folder = os.getcwd()
data_stats_folder = os.path.join(root_folder, 'images-data')


## Here: statistical test for our HOG analysis
Based on the data collected by the HOG_stats_{norm_block}_{pixels}.csv file

In [46]:
norm_block = None
pixels_side = 64
test_column = ['std. deviation']


filename = f"HOG_stats_{norm_block}_{pixels_side}pixels.csv"

df = pd.read_csv(os.path.join(data_stats_folder, filename))
print(df.columns)

df = df[df["image size"] == "(1920, 1920)"] # drop any smaller tests
df = df[['group', 'abs. deviation', 'std. deviation']] # focus on meaningful measurements

print("total amount of observations:", len(df))
print(df['group'].value_counts())

print(df['group'].unique())



Index(['Unnamed: 0', 'group', 'image size', 'pixels_per_cell',
       'elapsed_time (min:sec)', 'avg. direction', 'std. deviation',
       'abs. deviation', 'signal_threshold', 'signal_stats'],
      dtype='object')
total amount of observations: 60
group
CTR    30
MS     30
Name: count, dtype: int64
['CTR' 'MS']


###  In case there are only two groups, we run one single test ( paired, unpaired... to be discussed!)

In [55]:
ctr_group = df[df['group'] == 'CTR'][test_column].values.ravel()
ms_group = df[df['group'] == 'MS'][test_column].values.ravel()

# 1. Unpaired t-test (independent t-test)
# If the observations are independent, use the unpaired t-test

print(f"Group numerosity: CTR={len(ctr_group)}, MS={len(ms_group)}")
print(f"Group average of {test_column}: CTR={np.mean(ctr_group):.4f}, MS={np.mean(ms_group):4f}")

unpaired_t_stat, unpaired_t_p_value = stats.ttest_ind(ctr_group, ms_group)
print(f"Unpaired t-test: Statistic = {unpaired_t_stat:.4f}, p-value = {unpaired_t_p_value:.6f}")


Group numerosity: CTR=30, MS=30
Group average of ['std. deviation']: CTR=26.6752, MS=26.028100
Unpaired t-test: Statistic = 3.8841, p-value = 0.000266


In [56]:
# 2. Paired t-test
# Assuming you have paired observations, perform the paired t-test
paired_t_stat, paired_t_p_value = stats.ttest_rel(ctr_group, ms_group)
print(f"Paired t-test: Statistic = {paired_t_stat:.4f}, p-value = {paired_t_p_value:.6f}")

Paired t-test: Statistic = 5.3952, p-value = 0.000008


### In case multiple conditions are tested:
We perform Friedman-Nemenyi or some other test...